In [105]:
import pandas as pd
import re

# VARIABLES
ginger_annot = "references/ncbi_dataset/data/GCF_018446385.1/genomic_modified.gtf"
ath_annot = "references/ath/Phytozome/PhytozomeV12/Athaliana/annotation/Athaliana_167_TAIR10.annotation_info.txt"
blastp_g2a = "outputs/blastp/ginger_vs_ath.tsv"
final_csv = "outputs/rmd-output/Batch1.csv"

Goal: add arabidopsis best-hit to expression table generated using R

In [106]:
final_csv_df = pd.read_csv(final_csv, sep=",")
final_csv_df.head()

,Black_R1,Black_R2,Black_R3,Chitin_R1,Chitin_R2,Chitin_R3,Clean_R1,Clean_R2,Clean_R3,KHY26_R1,...,lfcSE.KHY26,stat.KHY26,pvalue.KHY26,padj.KHY26,baseMean.RKN,log2FoldChange.RKN,lfcSE.RKN,stat.RKN,pvalue.RKN,padj.RKN
F6E76_pgp001,299.37,149.46,44.86,15.53,11.00,41.21,174.06,148.47,328.29,42.66,...,0.460590,0.0,1.0,1.0,75.577937,0.508806,0.453865,0.0,1.0,1.0
F6E76_pgp002,29.32,20.27,2.37,4.18,1.86,5.83,15.61,28.95,15.17,2.06,...,0.756118,0.0,1.0,1.0,6.383657,0.393983,0.705686,0.0,1.0,1.0
F6E76_pgp003,152.93,128.76,11.35,11.27,6.61,25.49,143.15,132.36,113.61,7.43,...,0.504846,0.0,1.0,1.0,32.817504,-0.427678,0.487782,0.0,1.0,1.0
F6E76_pgp004,14.00,13.50,1.00,1.00,0.00,2.50,14.00,13.00,13.00,3.00,...,0.802628,0.0,1.0,1.0,3.663150,-0.410659,0.909689,0.0,1.0,1.0
F6E76_pgp005,146.74,86.71,10.87,4.26,4.82,19.94,99.43,72.06,180.08,19.44,...,0.600830,0.0,1.0,1.0,33.028656,0.511927,0.589614,0.0,1.0,1.0


Get Ginger "Gene ID -> Protein ID" info from GTF

In [25]:
ginger_annot_df = pd.read_csv(ginger_annot, sep="\t", header=None, comment="#")
print(ginger_annot_df.shape)
ginger_annot_df.head()

(1803194, 9)


,0,1,2,3,4,5,6,7,8
0,NC_055986.1,Gnomon,gene,5,1938,.,+,.,"gene_id ""LOC121992595""; transcript_id """"; db_x..."
1,NC_055986.1,Gnomon,transcript,5,1938,.,+,.,"gene_id ""LOC121992595""; transcript_id ""XM_0425..."
2,NC_055986.1,Gnomon,exon,5,37,.,+,.,"gene_id ""LOC121992595""; transcript_id ""XM_0425..."
3,NC_055986.1,Gnomon,exon,132,263,.,+,.,"gene_id ""LOC121992595""; transcript_id ""XM_0425..."
4,NC_055986.1,Gnomon,exon,352,426,.,+,.,"gene_id ""LOC121992595""; transcript_id ""XM_0425..."


In [46]:
ginger_p2g = {}
for attr in set(ginger_annot_df[8]):
    fields = [i.strip() for i in attr.strip().split(";")]
    attr_dict = {}
    for field in fields:
        key, *value = field.split(" ")
        value = " ".join(value).strip('"')
        attr_dict[key] = value
    if attr_dict.get("protein_id", False):
        ginger_p2g[attr_dict["protein_id"]] = attr_dict["gene_id"]

In [47]:
ginger_p2g

{'XP_042453704.1': 'LOC122038150',
 'XP_042458334.1': 'LOC122042342',
 'XP_042399183.1': 'LOC121989298',
 'XP_042462991.1': 'LOC122046377',
 'XP_042386496.1': 'LOC121978186',
 'XP_042431607.1': 'LOC122017948',
 'XP_042382840.1': 'LOC121975326',
 'XP_042431154.1': 'LOC122017557',
 'XP_042406600.1': 'LOC121996639',
 'XP_042418057.1': 'LOC122006571',
 'XP_042396571.1': 'LOC121986685',
 'XP_042450855.1': 'LOC122035807',
 'XP_042395449.1': 'LOC121985837',
 'XP_042390375.1': 'LOC121981742',
 'XP_042382446.1': 'LOC121975102',
 'XP_042448447.1': 'LOC122033481',
 'XP_042455919.1': 'LOC122040614',
 'XP_042385902.1': 'LOC121977384',
 'XP_042377529.1': 'LOC121970706',
 'XP_042446436.1': 'LOC122031385',
 'XP_042393936.1': 'LOC121984853',
 'XP_042456030.1': 'LOC122040676',
 'XP_042393155.1': 'LOC121984335',
 'XP_042399353.1': 'LOC121989395',
 'XP_042424819.1': 'LOC122012371',
 'XP_042385391.1': 'LOC121976993',
 'XP_042428406.1': 'LOC122015526',
 'XP_042433566.1': 'LOC122019964',
 'XP_042413462.1': '

In [53]:
ath_annot_df = pd.read_csv(ath_annot, sep="\t", header=0)
ath_annot_df.head()

,#pacId,locusName,transcriptName,peptideName,Pfam,Panther,KOG,KEGG/ec,KO,GO,Best-hit-rice-name,rice-symbol,rice-defline
0,19656964,AT1G01010,AT1G01010.1,AT1G01010.1,PF02365,"PTHR31989,PTHR31989:SF5",NaN,NaN,NaN,"GO:0006355,GO:0003677",LOC_Os08g44820.1,NaN,"no apical meristem protein, putative, expressed"
1,19655142,AT1G01020,AT1G01020.1,AT1G01020.1,PF04161,PTHR14467,KOG3134,NaN,NaN,NaN,LOC_Os03g57090.1,NaN,"arv1, putative, expressed"
2,19655143,AT1G01020,AT1G01020.2,AT1G01020.2,PF04161,PTHR14467,KOG3134,NaN,NaN,NaN,LOC_Os03g57090.1,NaN,"arv1, putative, expressed"
3,19649747,AT1G01030,AT1G01030.1,AT1G01030.1,PF02362,"PTHR31140,PTHR31140:SF4",NaN,NaN,NaN,GO:0003677,LOC_Os03g02900.1,NaN,"B3 DNA binding domain containing protein, expr..."
4,19652432,AT1G01040,AT1G01040.1,AT1G01040.1,"PF04851,PF14709,PF00271,PF00035,PF03368,PF0217...","PTHR14950,PTHR14950:SF3",NaN,3.1.26.3,K11592,"GO:0016787,GO:0005524,GO:0003677,GO:0016891,GO...",LOC_Os03g02970.1,NaN,"Dicer, putative, expressed"


In [57]:
ath_p2g = {}
for row in ath_annot_df.itertuples():
    ath_p2g[row.peptideName] = row.locusName
ath_p2g

{'AT1G01010.1': 'AT1G01010',
 'AT1G01020.1': 'AT1G01020',
 'AT1G01020.2': 'AT1G01020',
 'AT1G01030.1': 'AT1G01030',
 'AT1G01040.1': 'AT1G01040',
 'AT1G01040.2': 'AT1G01040',
 'AT1G01050.1': 'AT1G01050',
 'AT1G01060.1': 'AT1G01060',
 'AT1G01060.2': 'AT1G01060',
 'AT1G01060.3': 'AT1G01060',
 'AT1G01060.4': 'AT1G01060',
 'AT1G01060.5': 'AT1G01060',
 'AT1G01070.1': 'AT1G01070',
 'AT1G01070.2': 'AT1G01070',
 'AT1G01073.1': 'AT1G01073',
 'AT1G01080.1': 'AT1G01080',
 'AT1G01080.2': 'AT1G01080',
 'AT1G01090.1': 'AT1G01090',
 'AT1G01100.1': 'AT1G01100',
 'AT1G01100.2': 'AT1G01100',
 'AT1G01100.3': 'AT1G01100',
 'AT1G01100.4': 'AT1G01100',
 'AT1G01110.1': 'AT1G01110',
 'AT1G01110.2': 'AT1G01110',
 'AT1G01115.1': 'AT1G01115',
 'AT1G01120.1': 'AT1G01120',
 'AT1G01130.1': 'AT1G01130',
 'AT1G01140.1': 'AT1G01140',
 'AT1G01140.2': 'AT1G01140',
 'AT1G01140.3': 'AT1G01140',
 'AT1G01150.1': 'AT1G01150',
 'AT1G01160.1': 'AT1G01160',
 'AT1G01160.2': 'AT1G01160',
 'AT1G01170.1': 'AT1G01170',
 'AT1G01170.2'

In [70]:
blastp_g2a_df = pd.read_csv(blastp_g2a, sep="\t", header=None)
blastp_g2a_df["ath_pid"] = blastp_g2a_df[1].apply(lambda x: x.split("|")[0])
blastp_g2a_df["ath_gid"] = blastp_g2a_df["ath_pid"].apply(lambda x: ath_p2g.get(x, "NA"))
blastp_g2a_df["ginger_gid"] = blastp_g2a_df[0].apply(lambda x: ginger_p2g.get(x, "NA"))
print(blastp_g2a_df.shape)
blastp_g2a_df.head()

(112219, 15)


,0,1,2,3,4,5,6,7,8,9,10,11,ath_pid,ath_gid,ginger_gid
0,XP_042373031.1,AT3G50420.1|PACid:19662118,41.99,693,392,6,14,701,89,776,0.000000e+00,578.0,AT3G50420.1,AT3G50420,LOC122056748
1,XP_042373031.1,AT3G50420.1|PACid:19662118,26.91,550,372,9,55,587,22,558,8.000000e-57,207.0,AT3G50420.1,AT3G50420,LOC122056748
2,XP_042373032.1,AT2G25720.1|PACid:19642458,53.66,123,46,4,1,118,1,117,4.000000e-25,94.0,AT2G25720.1,AT2G25720,LOC121967087
3,XP_042373033.1,AT1G20080.1|PACid:19652904,70.45,538,157,2,88,625,1,536,0.000000e+00,798.0,AT1G20080.1,AT1G20080,LOC121967088
4,XP_042373034.1,AT1G20080.1|PACid:19652904,70.45,538,157,2,1,538,1,536,0.000000e+00,796.0,AT1G20080.1,AT1G20080,LOC121967088


In [63]:
gid_g2a = dict(zip(blastp_g2a_df["ginger_gid"], blastp_g2a_df["ath_gid"]))

In [64]:
final_csv_df.index.map(gid_g2a)

Index(['ATCG00020', 'ATCG00820', 'ATCG01310', 'ATCG01300', 'ATCG00860',
       'ATCG01250', 'ATCG01240', 'ATCG01130', 'ATCG01120', 'ATCG01110',
       ...
               nan, 'AT3G14470',         nan, 'AT3G14470', 'AT1G26330',
       'AT4G28250', 'AT1G20270', 'AT5G66150', 'AT4G38710', 'AT1G75800'],
      dtype='object', length=57020)

Add arabidopsis best hit

In [107]:
final_csv_df["ath_best_hit"] = final_csv_df.index.map(gid_g2a)
final_csv_df.head()

,Black_R1,Black_R2,Black_R3,Chitin_R1,Chitin_R2,Chitin_R3,Clean_R1,Clean_R2,Clean_R3,KHY26_R1,...,stat.KHY26,pvalue.KHY26,padj.KHY26,baseMean.RKN,log2FoldChange.RKN,lfcSE.RKN,stat.RKN,pvalue.RKN,padj.RKN,ath_best_hit
F6E76_pgp001,299.37,149.46,44.86,15.53,11.00,41.21,174.06,148.47,328.29,42.66,...,0.0,1.0,1.0,75.577937,0.508806,0.453865,0.0,1.0,1.0,ATCG00020
F6E76_pgp002,29.32,20.27,2.37,4.18,1.86,5.83,15.61,28.95,15.17,2.06,...,0.0,1.0,1.0,6.383657,0.393983,0.705686,0.0,1.0,1.0,ATCG00820
F6E76_pgp003,152.93,128.76,11.35,11.27,6.61,25.49,143.15,132.36,113.61,7.43,...,0.0,1.0,1.0,32.817504,-0.427678,0.487782,0.0,1.0,1.0,ATCG01310
F6E76_pgp004,14.00,13.50,1.00,1.00,0.00,2.50,14.00,13.00,13.00,3.00,...,0.0,1.0,1.0,3.663150,-0.410659,0.909689,0.0,1.0,1.0,ATCG01300
F6E76_pgp005,146.74,86.71,10.87,4.26,4.82,19.94,99.43,72.06,180.08,19.44,...,0.0,1.0,1.0,33.028656,0.511927,0.589614,0.0,1.0,1.0,ATCG00860


In [95]:
# for g in ["CK", "LETC", "LITR"]:
#     if g == "LITR":
#         final_csv_df[f"baseMean.{g}"] = final_csv_df[[f"{g}_R1", f"{g}_R3"]].mean(axis=1)
#     else:      
#         final_csv_df[f"baseMean.{g}"] = final_csv_df[[f"{g}_R1", f"{g}_R2", f"{g}_R3"]].mean(axis=1)

In [108]:
final_csv_df.head()

,Black_R1,Black_R2,Black_R3,Chitin_R1,Chitin_R2,Chitin_R3,Clean_R1,Clean_R2,Clean_R3,KHY26_R1,...,stat.KHY26,pvalue.KHY26,padj.KHY26,baseMean.RKN,log2FoldChange.RKN,lfcSE.RKN,stat.RKN,pvalue.RKN,padj.RKN,ath_best_hit
F6E76_pgp001,299.37,149.46,44.86,15.53,11.00,41.21,174.06,148.47,328.29,42.66,...,0.0,1.0,1.0,75.577937,0.508806,0.453865,0.0,1.0,1.0,ATCG00020
F6E76_pgp002,29.32,20.27,2.37,4.18,1.86,5.83,15.61,28.95,15.17,2.06,...,0.0,1.0,1.0,6.383657,0.393983,0.705686,0.0,1.0,1.0,ATCG00820
F6E76_pgp003,152.93,128.76,11.35,11.27,6.61,25.49,143.15,132.36,113.61,7.43,...,0.0,1.0,1.0,32.817504,-0.427678,0.487782,0.0,1.0,1.0,ATCG01310
F6E76_pgp004,14.00,13.50,1.00,1.00,0.00,2.50,14.00,13.00,13.00,3.00,...,0.0,1.0,1.0,3.663150,-0.410659,0.909689,0.0,1.0,1.0,ATCG01300
F6E76_pgp005,146.74,86.71,10.87,4.26,4.82,19.94,99.43,72.06,180.08,19.44,...,0.0,1.0,1.0,33.028656,0.511927,0.589614,0.0,1.0,1.0,ATCG00860


In [109]:
final_csv_df["gene_id"] = final_csv_df.index
final_csv_df.to_csv("outputs/rmd-output/Batch1_ath_best_hit.csv", index=False)